In [1]:
import pandas as pd
import numpy as np
import pymongo
from pymongo import MongoClient
connection = MongoClient()

In [2]:
n_words_in_dict = 1000 # Number of features
n_obs = 3000 # Number of total observations to consider
split_perc = 0.8 # Training percentage
seed = 200 # Seed for the extraction initialisation
db = connection.MPST

plots_df = pd.DataFrame(db.plots.find({}))
tags_df = pd.DataFrame(db.tags.find({}))
total_df = plots_df.merge(tags_df, on='_id', how='inner')
for col in total_df:
    total_df[col] = total_df[col].astype('string')
print('Plots dimension:', plots_df.shape)
print('Tags dimension:', tags_df.shape)
total_df

Plots dimension: (14828, 3)
Tags dimension: (14828, 2)


_id                                          title  \
0      tt0057603                        I tre volti della paura   
1      tt1733125  Dungeons & Dragons: The Book of Vile Darkness   
2      tt0033045                     The Shop Around the Corner   
3      tt0113862                             Mr. Holland's Opus   
4      tt0086250                                       Scarface   
...          ...                                            ...   
14823  tt0219952                                  Lucky Numbers   
14824  tt1371159                                     Iron Man 2   
14825  tt0063443                                     Play Dirty   
14826  tt0039464                                      High Wall   
14827  tt0235166                               Against All Hope   

                                           plot_synopsis  \
0      Note: this synopsis is for the orginal Italian...   
1      Two thousand years ago, Nhagruul the Foul, a s...   
2      Matuschek's, a gift store in Budapest, is the ...   
3      Glenn Holland, not a morning person by anyone'...   
4      In May 1980, a Cuban man named Tony Montana (A...   
...                                                  ...   
14823  In 1988 Russ Richards (John Travolta), the wea...   
14824  In Russia, the media covers Tony Stark's discl...   
14825  During the North African Campaign in World War...   
14826  Steven Kenet catches his unfaithful wife in th...   
14827  Sometime in the 1950s in Chicago a man, Cecil ...   

                                                    tags  
0              cult, horror, gothic, murder, atmospheric  
1                                               violence  
2                                               romantic  
3                 inspiring, romantic, stupid, feel-good  
4      cruelty, murder, dramatic, cult, violence, atm...  
...                                                  ...  
14823                                     comedy, murder  
14824                         good versus evil, violence  
14825                                           anti war  
14826                                             murder  
14827                                     christian film  

[14828 rows x 4 columns]

In [3]:
tagged_df = total_df.copy()
tags_col = []

for i in range(tagged_df.shape[0]):
    tags_col.append(set(tagged_df.iloc[i, 3].split(', ')))

tagged_df['tags'] = tags_col
tagged_df

_id                                          title  \
0      tt0057603                        I tre volti della paura   
1      tt1733125  Dungeons & Dragons: The Book of Vile Darkness   
2      tt0033045                     The Shop Around the Corner   
3      tt0113862                             Mr. Holland's Opus   
4      tt0086250                                       Scarface   
...          ...                                            ...   
14823  tt0219952                                  Lucky Numbers   
14824  tt1371159                                     Iron Man 2   
14825  tt0063443                                     Play Dirty   
14826  tt0039464                                      High Wall   
14827  tt0235166                               Against All Hope   

                                           plot_synopsis  \
0      Note: this synopsis is for the orginal Italian...   
1      Two thousand years ago, Nhagruul the Foul, a s...   
2      Matuschek's, a gift store in Budapest, is the ...   
3      Glenn Holland, not a morning person by anyone'...   
4      In May 1980, a Cuban man named Tony Montana (A...   
...                                                  ...   
14823  In 1988 Russ Richards (John Travolta), the wea...   
14824  In Russia, the media covers Tony Stark's discl...   
14825  During the North African Campaign in World War...   
14826  Steven Kenet catches his unfaithful wife in th...   
14827  Sometime in the 1950s in Chicago a man, Cecil ...   

                                                    tags  
0            {horror, murder, cult, gothic, atmospheric}  
1                                             {violence}  
2                                             {romantic}  
3               {stupid, romantic, feel-good, inspiring}  
4      {dramatic, violence, murder, romantic, atmosph...  
...                                                  ...  
14823                                   {murder, comedy}  
14824                       {violence, good versus evil}  
14825                                         {anti war}  
14826                                           {murder}  
14827                                   {christian film}  

[14828 rows x 4 columns]

In [4]:
mur_movies = tagged_df[['murder' in tags for tags in tagged_df['tags']]]
rom_movies = tagged_df[['romantic' in tags for tags in tagged_df['tags']]]
common_movies = mur_movies.merge(rom_movies, on='_id', how='inner')

mur_rom_movies = pd.concat([mur_movies, rom_movies])
mur_rom_movies = mur_rom_movies[~mur_rom_movies['_id'].isin(common_movies['_id'])]
mur_rom_movies

_id                    title  \
0      tt0057603  I tre volti della paura   
7      tt0408790               Flightplan   
9      tt1615065                  Savages   
11     tt0078908                The Brood   
12     tt0795493        Cassandra's Dream   
...          ...                      ...   
14792  tt0061209     You're a Big Boy Now   
14803  tt0028010           My Man Godfrey   
14810  tt2209418          Before Midnight   
14817  tt0028522              Yellow Dust   
14821  tt0025601        One Night of Love   

                                           plot_synopsis  \
0      Note: this synopsis is for the orginal Italian...   
7      Kyle Pratt (Jodie Foster) is a propulsion engi...   
9      The movie begins with a video being shot of me...   
11     At the Somafree Institute, Dr. Hal Raglan humi...   
12     Brothers Terry (Colin Farrell) and Ian (Ewan M...   
...                                                  ...   
14792  Bernard Chanticleer (Peter Kastner), called "B...   
14803  During the Great Depression, Godfrey "Smith" P...   
14810  Nine years have passed since Before Sunset. Je...   
14817  The film opens in the hills of California. Bob...   
14821  Opera singer Mary Barrett (Grace Moore) leaves...   

                                                    tags  
0            {horror, murder, cult, gothic, atmospheric}  
7      {flashback, murder, mystery, action, suspenseful}  
9       {flashback, violence, murder, revenge, neo noir}  
11                 {murder, cult, violence, psychedelic}  
12                           {tragedy, dramatic, murder}  
...                                                  ...  
14792                                   {romantic, cult}  
14803             {satire, romantic, depressing, comedy}  
14810  {storytelling, romantic, fantasy, philosophica...  
14817           {romantic, historical fiction, violence}  
14821                                         {romantic}  

[6998 rows x 4 columns]

In [5]:
# Murder = 1; Romantic = 0
clean_tags_df = mur_rom_movies.copy()

for i in range(clean_tags_df.shape[0]):
    if ('murder' in clean_tags_df.iloc[i, 3]):
        clean_tags_df.iloc[i, 3] = 1
    else:
        clean_tags_df.iloc[i, 3] = 0

clean_tags_df['tags'] = clean_tags_df['tags'].astype('int64')
clean_tags_df

_id                    title  \
0      tt0057603  I tre volti della paura   
7      tt0408790               Flightplan   
9      tt1615065                  Savages   
11     tt0078908                The Brood   
12     tt0795493        Cassandra's Dream   
...          ...                      ...   
14792  tt0061209     You're a Big Boy Now   
14803  tt0028010           My Man Godfrey   
14810  tt2209418          Before Midnight   
14817  tt0028522              Yellow Dust   
14821  tt0025601        One Night of Love   

                                           plot_synopsis  tags  
0      Note: this synopsis is for the orginal Italian...     1  
7      Kyle Pratt (Jodie Foster) is a propulsion engi...     1  
9      The movie begins with a video being shot of me...     1  
11     At the Somafree Institute, Dr. Hal Raglan humi...     1  
12     Brothers Terry (Colin Farrell) and Ian (Ewan M...     1  
...                                                  ...   ...  
14792  Bernard Chanticleer (Peter Kastner), called "B...     0  
14803  During the Great Depression, Godfrey "Smith" P...     0  
14810  Nine years have passed since Before Sunset. Je...     0  
14817  The film opens in the hills of California. Bob...     0  
14821  Opera singer Mary Barrett (Grace Moore) leaves...     0  

[6998 rows x 4 columns]

In [6]:
plotted_df = clean_tags_df.copy()
plots_array = []

for i in range(plotted_df.shape[0]):
    i_plot = plotted_df.iloc[i, 2].lower()
    i_plot = i_plot.replace(':', ' ')
    i_plot = i_plot.replace(';', ' ')
    i_plot = i_plot.replace(',', ' ')
    i_plot = i_plot.replace('.', ' ')
    i_plot = i_plot.replace('/', ' ')
    i_plot = i_plot.replace('\\', ' ')
    i_plot = i_plot.replace('(', ' ')
    i_plot = i_plot.replace(')', ' ')
    i_plot = i_plot.replace('[', ' ')
    i_plot = i_plot.replace(']', ' ')
    i_plot = i_plot.replace('{', ' ')
    i_plot = i_plot.replace('}', ' ')
    i_plot = i_plot.replace('!', ' ')
    i_plot = i_plot.replace('"', ' ')
    plots_array.append(i_plot)

if n_obs > plotted_df.shape[0]:
    n_obs = plotted_df.shape[0]
frac_val = n_obs / plotted_df.shape[0]
plotted_df['plot_synopsis'] = plots_array
plotted_df = plotted_df.groupby('tags').apply(lambda x: x.sample(random_state=seed, frac=frac_val)).reset_index(drop=True)
plotted_df

_id                                          title  \
0     tt0082861                                        Othello   
1     tt1341341                                       Ceremony   
2     tt0173102                        Raju Ban Gaya Gentleman   
3     tt0064449                         How to Commit Marriage   
4     tt0081534                              Somewhere in Time   
...         ...                                            ...   
5995  tt0099850                               Internal Affairs   
5996  tt0068341                             The Valachi Papers   
5997  tt0094761                                       The Blob   
5998  tt1146438  The Librarian: The Curse of the Judas Chalice   
5999  tt0444682                                    The Reaping   

                                          plot_synopsis  tags  
0     === act i ===\nroderigo  a poor and dissolute ...     0  
1     the film begins with masuo sakurada receiving ...     0  
2     raj mathur  shahrukh khan  is a young engineer...     0  
3     nancy  the 19-year-old daughter of frank and e...     0  
4     on may 19  1972  college theatre student richa...     0  
...                                                 ...   ...  
5995  during a drug bust  lapd patrolmen dennis peck...     1  
5996  the movie begins in the atlanta federal penite...     1  
5997  during one long night in a small rural pennsyl...     1  
5998  glenn estate auction in london \nflynn primps ...     1  
5999  father costigan  stephen rea  find that his ph...     1  

[6000 rows x 4 columns]

In [7]:
training_df = plotted_df.copy()
training_df = training_df.groupby('tags').apply(lambda x: x.sample(random_state=seed, frac=split_perc))
training_df = training_df.sample(random_state=seed, frac=1).reset_index(drop=True)

validation_df = plotted_df[~plotted_df['_id'].isin(training_df['_id'])]
validation_df = validation_df.sample(random_state=seed, frac=1).reset_index(drop=True)
validation_df

_id                 title  \
0     tt1596365    The Woman in Black   
1     tt0081491                 Shaan   
2     tt0100935         Wild at Heart   
3     tt0067309                 Klute   
4     tt0452608            Death Race   
...         ...                   ...   
1195  tt0047411  Ride Clear of Diablo   
1196  tt0272688                Kasoor   
1197  tt0028096  The Petrified Forest   
1198  tt0060748          Nevada Smith   
1199  tt0203119            Sexy Beast   

                                          plot_synopsis  tags  
0     arthur kipps  daniel radcliffe  is a young law...     1  
1     dcp  deputy commissioner of police  shiv kumar...     1  
2     at a formal dinner party in cape fear  north c...     1  
3     the film begins with the disappearance of penn...     1  
4     in 2012  the collapse of the us economy and th...     1  
...                                                 ...   ...  
1195  sheriff fred  paul birch  and lawyer tom  will...     1  
1196  the film starts with the murder of priti  wife...     1  
1197  in the midst of the great depression  alan squ...     1  
1198  in the west of the 1890s  a trio of outlaws  b...     1  
1199  ex-con and expert safe-cracker gary  gal  dove...     1  

[1200 rows x 4 columns]

In [8]:
stopwords = set()
for line in open('stopwords.txt'):
    stopwords.add(line.strip())
stopwords

{'a',
 'about',
 'above',
 'across',
 'after',
 'afterwards',
 'again',
 'against',
 'all',
 'almost',
 'alone',
 'along',
 'already',
 'also',
 'although',
 'always',
 'am',
 'among',
 'amongst',
 'amoungst',
 'amount',
 'an',
 'and',
 'another',
 'any',
 'anyhow',
 'anyone',
 'anything',
 'anyway',
 'anywhere',
 'are',
 'around',
 'as',
 'at',
 'back',
 'be',
 'became',
 'because',
 'become',
 'becomes',
 'becoming',
 'been',
 'before',
 'beforehand',
 'behind',
 'being',
 'below',
 'beside',
 'besides',
 'between',
 'beyond',
 'bill',
 'both',
 'bottom',
 'but',
 'by',
 'call',
 'can',
 'cannot',
 'cant',
 'co',
 'computer',
 'con',
 'could',
 'couldnt',
 'cry',
 'de',
 'describe',
 'detail',
 'do',
 'done',
 'down',
 'due',
 'during',
 'each',
 'eg',
 'eight',
 'either',
 'eleven',
 'else',
 'elsewhere',
 'empty',
 'enough',
 'etc',
 'even',
 'ever',
 'every',
 'everyone',
 'everything',
 'everywhere',
 'except',
 'few',
 'fifteen',
 'fify',
 'fill',
 'find',
 'fire',
 'first',
 'f

In [9]:
from sklearn import feature_extraction
tfidf_vect = feature_extraction.text.TfidfVectorizer(max_features=n_words_in_dict, stop_words=stopwords)

tfidf_result_tr = tfidf_vect.fit_transform(training_df['plot_synopsis'].values)
tfidf_result_val = tfidf_vect.fit_transform(validation_df['plot_synopsis'].values)

In [10]:
tfidf_vect.get_feature_names()

['000',
 '10',
 'abandoned',
 'able',
 'accepts',
 'accident',
 'accidentally',
 'ace',
 'act',
 'actually',
 'adam',
 'address',
 'admits',
 'affair',
 'age',
 'agent',
 'agents',
 'ago',
 'agree',
 'agrees',
 'air',
 'airport',
 'alex',
 'alice',
 'alien',
 'alive',
 'allison',
 'american',
 'amy',
 'andrew',
 'angry',
 'ann',
 'anna',
 'anne',
 'announces',
 'answer',
 'answers',
 'anton',
 'apartment',
 'apparently',
 'appear',
 'appears',
 'approaches',
 'area',
 'arm',
 'arms',
 'army',
 'arrest',
 'arrested',
 'arrive',
 'arrives',
 'arthur',
 'ash',
 'ask',
 'asked',
 'asking',
 'asks',
 'assistant',
 'attack',
 'attacked',
 'attacks',
 'attempt',
 'attempts',
 'attention',
 'audience',
 'aunt',
 'avoid',
 'away',
 'baby',
 'bad',
 'bag',
 'band',
 'bane',
 'bank',
 'bar',
 'basement',
 'bathroom',
 'batman',
 'battle',
 'beach',
 'beautiful',
 'bed',
 'bedroom',
 'begin',
 'beginning',
 'begins',
 'believe',
 'believes',
 'believing',
 'ben',
 'best',
 'better',
 'big',
 'bill

In [11]:
tfidf_df_tr = pd.DataFrame(tfidf_result_tr.todense(), columns=tfidf_vect.get_feature_names())
tfidf_df_tr['romantic_murder'] = training_df['tags'].values
tfidf_df_val = pd.DataFrame(tfidf_result_val.todense(), columns=tfidf_vect.get_feature_names())
tfidf_df_val['romantic_murder'] = validation_df['tags'].values
tfidf_df_tr

000   10  abandoned  able  accepts  accident  accidentally  ace  act  \
0     0.0  0.0   0.000000   0.0      0.0  0.000000      0.000000  0.0  0.0   
1     0.0  0.0   0.031735   0.0      0.0  0.000000      0.000000  0.0  0.0   
2     0.0  0.0   0.000000   0.0      0.0  0.000000      0.065732  0.0  0.0   
3     0.0  0.0   0.000000   0.0      0.0  0.000000      0.000000  0.0  0.0   
4     0.0  0.0   0.063635   0.0      0.0  0.000000      0.000000  0.0  0.0   
...   ...  ...        ...   ...      ...       ...           ...  ...  ...   
4795  0.0  0.0   0.000000   0.0      0.0  0.022899      0.000000  0.0  0.0   
4796  0.0  0.0   0.000000   0.0      0.0  0.000000      0.000000  0.0  0.0   
4797  0.0  0.0   0.000000   0.0      0.0  0.000000      0.000000  0.0  0.0   
4798  0.0  0.0   0.019918   0.0      0.0  0.000000      0.000000  0.0  0.0   
4799  0.0  0.0   0.000000   0.0      0.0  0.000000      0.000000  0.0  0.0   

      actually  ...   written  wrong  year  years        yi      york  \
0     0.066707  ...  0.000000    0.0   0.0    0.0  0.056037  0.000000   
1     0.000000  ...  0.000000    0.0   0.0    0.0  0.062606  0.000000   
2     0.000000  ...  0.000000    0.0   0.0    0.0  0.000000  0.021219   
3     0.000000  ...  0.000000    0.0   0.0    0.0  0.000000  0.000000   
4     0.000000  ...  0.000000    0.0   0.0    0.0  0.125537  0.000000   
...        ...  ...       ...    ...   ...    ...       ...       ...   
4795  0.022284  ...  0.000000    0.0   0.0    0.0  0.018719  0.000000   
4796  0.000000  ...  0.000000    0.0   0.0    0.0  0.000000  0.000000   
4797  0.000000  ...  0.053987    0.0   0.0    0.0  0.051764  0.000000   
4798  0.000000  ...  0.000000    0.0   0.0    0.0  0.000000  0.016059   
4799  0.000000  ...  0.000000    0.0   0.0    0.0  0.000000  0.000000   

         young   younger      zach  romantic_murder  
0     0.000000  0.000000  0.074276                1  
1     0.000000  0.024672  0.000000                1  
2     0.000000  0.010230  0.000000                0  
3     0.129174  0.000000  0.069350                1  
4     0.000000  0.049472  0.000000                0  
...        ...       ...       ...              ...  
4795  0.000000  0.000000  0.000000                0  
4796  0.000000  0.000000  0.000000                1  
4797  0.000000  0.000000  0.068611                0  
4798  0.000000  0.015485  0.000000                1  
4799  0.000000  0.000000  0.135857                1  

[4800 rows x 1001 columns]

In [12]:
tfidf_df_tr.to_csv('moviedata_training_3000.csv', index=False)
tfidf_df_val.to_csv('moviedata_validation_3000.csv', index=False)
connection.close()